In [1]:
import numpy as np
from skimage.transform import resize
import pylidc as pl
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)
n = scans.count()

In [3]:
image_size = (256, 256)

In [12]:
def get_data(i, only_nodules=True):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    if only_nodules:
        indices = [l.sum() > 0 for l in y]
        X = X[indices]
        y = y[indices]
    y = np.expand_dims(y, axis=-1)
    return X, y

In [5]:
model = UNet((*image_size, 1), batchnorm=True, dropout=False)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
for _ in range(25):
    while True:
        i = np.random.randint(n)
        print(i)
        try:
            X, y = get_data(i)
            try:
                model.fit(X, y, batch_size=1, epochs=2)
            except AttributeError:
                model.fit(X, y, batch_size=1, epochs=2, verbose=0)
            break
        except OSError:
            print('Problem getting data')

399
Epoch 1/2
8/8 [==============================] - 3s 350ms/step - loss: 0.0437 - acc: 0.9997
Epoch 2/2
8/8 [==============================] - 3s 313ms/step - loss: 0.0400 - acc: 0.9998
642
Epoch 1/2
4/4 [==============================] - 1s 315ms/step - loss: 0.0382 - acc: 0.9996
Epoch 2/2
4/4 [==============================] - 1s 308ms/step - loss: 0.0367 - acc: 0.9996
717
Epoch 1/2
58
Epoch 1/2
15/15 [==============================] - 5s 332ms/step - loss: 0.0340 - acc: 0.9995
Epoch 2/2
15/15 [==============================] - 5s 312ms/step - loss: 0.0296 - acc: 0.9995
522
Epoch 1/2
5/5 [==============================] - 2s 308ms/step - loss: 0.0269 - acc: 0.9997
Epoch 2/2
5/5 [==============================] - 2s 310ms/step - loss: 0.0257 - acc: 0.9997
985
Epoch 1/2
8/8 [==============================] - 3s 357ms/step - loss: 0.0248 - acc: 0.9998
Epoch 2/2
8/8 [==============================] - 2s 307ms/step - loss: 0.0228 - acc: 0.9998
136
Epoch 1/2
22/22 [======================

In [11]:
model.save('unet.hdf5')